In [2]:
from __future__ import unicode_literals, print_function
from pathlib import Path
from spacy import displacy
from spacy.util import minibatch, compounding
from spacy.pipeline import EntityRuler    
from spacy.tokens import Span
import plac                                     #Paquetes propios de SpaCy
import random                    #Aleatorizará los textos de entrenamiento
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
import datetime as dt  
import os 

### Lo que tengo que intentar conseguir ahora es que este bucle itere sobre todos los textos (y anotaciones) del training y del development set



In [41]:
#Inicializo la lista que va a contener todos los textos pertenecientes al fold:
train_data=[]

#Bucle con el que voy a iterar sobre todos los textos y anotaciones de una epoca de entrenamiento:
for it in range(3,12):
    
    txt_path = '/Users/ignaciobacierofernandez/Desktop/cantemist/dev_set/txt/dev_' + str(it) + '.txt' #/Users/ignaciobacierofernandez/Desktop/cantemist/train-set-to-publish/cantemist-ner #/Users/ignaciobacierofernandez/Desktop/cantemist/dev_set/
    txt = open(txt_path, "r")

    texto = txt.read()
    texto = texto.replace('\n', ' ') #Quito etiquetas '\n'
    #Antes habia convertido los dobles espacios en espacios simples creyendo que eran por el proceso de limpieza y no, es porque no saben escribir.

    annot_path = '/Users/ignaciobacierofernandez/Desktop/cantemist/dev_set/ann/dev_' + str(it) + '.ann'
    annot = []

    with open(annot_path) as an:

        for line in an.readlines():
            line = line.split()
            data = (int(line[2]), int(line[3]), line[1])
            annot.append(data)

    #         print(annot)

    # entities
    entities = {"entities": annot}

    #Train data 'i'
    dev_i = (texto,entities) #Esta lista debe contener no solo un texto sino todos los textos del fold

    #Lista de los datos de entrenamiento en el formato sugerido por spaCy
    train_data.append(dev_i)
    
# print(train_data)

### Para comprobar que he hecho las transformaciones de los documentos bien: 

    Cojo al azar cualquier texto y compruebo que las anotaciones del archivo de anotacion asociado concuerdan con la respectiva entidad anotada
    
    HECHO

Tengo que contar el numero de etiquetas \n que quito porque cada una de ellas me descoloca en 1 los offsets de las anotaciones.

### Bucle de entrenamiento 

    Empiezo con el modelo pre-entrenado "es_core_news_md"

In [6]:
def training_loop(model=None,new_model_name=None,output_dir=None,n_iter=None):  
   
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("es")  # create blank Language class
        print("Created blank 'es' model")

    # add new  built-in pipeline components 
    # nlp.create_pipe works for built-ins that are registered with spaCy
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
        
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        i=0
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            i+=1
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes=[pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):     #Entrenamiento del modulo NER
    
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.resume_training()
        for itn in range(n_iter):
            random.shuffle(train_data)
            losses = {}  
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))    
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,          # batch of texts
                    annotations,    # batch of annotations
                    drop= 0.15,     # dropout-make it harder to memorise data 
                    sgd=optimizer,  # callable to update weights
                    losses=losses)

            print(f"Losses at iteration {itn} - {dt.datetime.now()}", losses)
            
    # test the trained model
    for text, _ in train_data:
        doc = nlp(text)
        displacy.render(doc, style="ent", jupyter=True)
        print('')
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)  # rename model

    # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in train_data:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

In [40]:
model = '/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_4'   #es_core_news_md
new_model = 'it_5'
output_dir = '/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5'     
n_it = 30

#Llamada a la funcion training_loop

training_loop( model, new_model, output_dir, n_it)     

Loaded model '/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_4'
Losses at iteration 0 - 2020-08-03 18:49:45.447244 {'ner': 3071.439663250907}
Losses at iteration 1 - 2020-08-03 18:50:04.970457 {'ner': 2629.2293603865214}
Losses at iteration 2 - 2020-08-03 18:50:25.628073 {'ner': 2081.0493427546558}
Losses at iteration 3 - 2020-08-03 18:50:48.595366 {'ner': 2143.8573556226165}
Losses at iteration 4 - 2020-08-03 18:51:13.016534 {'ner': 2105.8253823884734}
Losses at iteration 5 - 2020-08-03 18:51:36.517642 {'ner': 1971.5449753832593}
Losses at iteration 6 - 2020-08-03 18:51:59.964819 {'ner': 1804.691106650522}
Losses at iteration 7 - 2020-08-03 18:52:23.239992 {'ner': 1993.6985976652704}
Losses at iteration 8 - 2020-08-03 18:52:46.788100 {'ner': 1949.3030490824237}
Losses at iteration 9 - 2020-08-03 18:53:10.315905 {'ner': 1794.299571965205}
Losses at iteration 10 - 2020-08-03 18:53:33.642429 {'ner': 1400.2185309945476}
Losses at iteration 11 - 2020-08-03 18


Saved model to /Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5
Loading from /Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5
Entities [('lesión cutánea', 'MORFOLOGIA_NEOPLASIA'), ('nódulo subcutáneo', 'MORFOLOGIA_NEOPLASIA'), ('nódulo subcutáneo', 'MORFOLOGIA_NEOPLASIA'), ('carcinoma de células claras', 'MORFOLOGIA_NEOPLASIA'), ('nódulo subcutáneo', 'MORFOLOGIA_NEOPLASIA'), ('metastásico', 'MORFOLOGIA_NEOPLASIA'), ('nódulos pulmonares', 'MORFOLOGIA_NEOPLASIA'), ('metástasis', 'MORFOLOGIA_NEOPLASIA'), ('metástasis', 'MORFOLOGIA_NEOPLASIA'), ('carcinoma renal de células claras (Fuhrman grado 1) de 9 cm (pT3a pN0 pM1)', 'MORFOLOGIA_NEOPLASIA'), ('carcinoma renal de células claras metastásico', 'MORFOLOGIA_NEOPLASIA'), ('afectación pulmonar', 'MORFOLOGIA_NEOPLASIA'), ('progresión pulmonar', 'MORFOLOGIA_NEOPLASIA'), ('nódulos pulmonares', 'MORFOLOGIA_NEOPLASIA'), ('metastásicos', 'MORFOLOGIA_NEOPLASIA'), ('nódulos pulmonares', 

### Evaluacion del modelo

In [10]:
def evaluate(model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [42]:
nlp = spacy.load('/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5')
results_all = evaluate(nlp, train_data)   
print(results_all)   

{'uas': 0.0, 'las': 0.0, 'ents_p': 83.22981366459628, 'ents_r': 83.75, 'ents_f': 83.48909657320873, 'ents_per_type': {'MORFOLOGIA_NEOPLASIA': {'p': 83.22981366459628, 'r': 83.75, 'f': 83.48909657320873}}, 'tags_acc': 0.0, 'token_acc': 100.0}


In [130]:
# txt_path = "/Users/ignaciobacierofernandez/Desktop/cantemist/dev_set/txt/dev_485.txt"
# txt = open(txt_path, "r")
# texto = txt.read()
# texto = texto.replace('\n', ' ') #Quito etiquetas '\n'
# texto = texto.replace('  ', ' ') #Se me han creado dobles espacios donde antes estaban estas etiquetas, los convierto en espacios simples
txt_path = "/Users/ignaciobacierofernandez/Desktop/cantemist/dev_set/txt/dev_66.txt"
txt = open(txt_path, "r")
txt.read()[5152:1111]

'metastásica visceral del carcinoma de células de Merkel'

### Aqui es donde tengo que crear los documentos una vez predichas todas las entidades del test set

In [10]:
nlp = spacy.load("/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5")
for m in range(100015,900015): 
    for n in range (1, 21):
        txt_path = '/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set/S0210-48062005000'+str(m)+'-'+str(n)+'.txt'  #Comprobar el path #/Users/ignaciobacierofernandez/Desktop/cantemist/test-background-set-to-publish
        if os.path.isfile(txt_path):
            txt = open(txt_path, "r")
            texto = txt.read()   #/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set/caso_clinico_radiologia1.txt
            doc = nlp(texto)
            with open('/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/ann/S0210-48062005000'+str(m)+'-'+str(n)+'.ann', 'w') as anot:  #Cambiar por el nombre del archivo que estemos prediciendo
                i=1
                for ent in doc.ents:
                    anot.write('T'+str(i)+'\t'+ent.label_+' '+str(ent.start_char)+' '+str(ent.end_char)+'\t'+ent.text+'\n')
                    i+=1
        else:
            continue
   
    #Creamos el archivo de anotacion
    

    

In [6]:
nlp = spacy.load("/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5")
for m in range(4,5): 
    for n in range (1, 21):
        txt_path = '/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set/S0120-4157201500040000'+str(m)+'-'+str(n)+'.txt'  #Comprobar el path #/Users/ignaciobacierofernandez/Desktop/cantemist/test-background-set-to-publish
        if os.path.isfile(txt_path):
            txt = open(txt_path, "r")
            texto = txt.read()   #/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set/caso_clinico_radiologia1.txt
            doc = nlp(texto)
            with open('/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/ann/S0120-4157201500040000'+str(m)+'-'+str(n)+'.ann', 'w') as anot:  #Cambiar por el nombre del archivo que estemos prediciendo
                i=1
                for ent in doc.ents:
                    anot.write('T'+str(i)+'\t'+ent.label_+' '+str(ent.start_char)+' '+str(ent.end_char)+'\t'+ent.text+'\n')
                    i+=1
        else:
            continue
   
    #Creamos el archivo de anotacion
    

    

In [20]:
nlp = spacy.load("/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/it_5")

path = '/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set'
files = os.listdir(path)
files = sorted(files)

for f in files:
   
    txt_path = '/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set/'+str(f)
    if os.path.isfile(txt_path):
        txt = open(txt_path, "r")
        texto = txt.read()   #/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set/caso_clinico_radiologia1.txt
        doc = nlp(texto)
        f = f[:-4]
        f = f + '.ann'
        with open('/Users/ignaciobacierofernandez/Desktop/cantemist/model/1-cantemist_baciero/ann_num/'+str(f), 'w') as anot:  #Cambiar por el nombre del archivo que estemos prediciendo
            i=1
            for ent in doc.ents:
                anot.write('T'+str(i)+'\t'+ent.label_+' '+str(ent.start_char)+' '+str(ent.end_char)+'\t'+ent.text+'\n')
                i+=1
    else:
        continue
  

In [19]:
path = '/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set'
files = os.listdir(path)
files = sorted(files)

for f in files:
    f = f[:-4]
    f = f + '.ann'
    print(f)

S0004-06142005000100009-1.ann
S0004-06142005000200004-1.ann
S0004-06142005000200009-1.ann
S0004-06142005000200009-2.ann
S0004-06142005000200009-3.ann
S0004-06142005000200009-4.ann
S0004-06142005000200009-5.ann
S0004-06142005000300011-1.ann
S0004-06142005000300012-1.ann
S0004-06142005000400011-1.ann
S0004-06142005000500011-1.ann
S0004-06142005000500014-1.ann
S0004-06142005000700014-1.ann
S0004-06142005000900012-1.ann
S0004-06142005000900013-1.ann
S0004-06142005000900014-1.ann
S0004-06142005000900015-1.ann
S0004-06142005000900016-1.ann
S0004-06142005000900017-1.ann
S0004-06142005001000010-1.ann
S0004-06142005001000011-1.ann
S0004-06142005001000011-2.ann
S0004-06142005001000011-3.ann
S0004-06142005001000014-1.ann
S0004-06142005001000015-1.ann
S0004-06142005001000016-1.ann
S0004-06142006000100010-1.ann
S0004-06142006000100011-1.ann
S0004-06142006000100012-1.ann
S0004-06142006000100013-1.ann
S0004-06142006000100014-1.ann
S0004-06142006000100015-1.ann
S0004-06142006000100015-2.ann
S0004-0614

In [16]:
path = '/Users/ignaciobacierofernandez/Desktop/cantemist_backup/test-background-set'
files = os.listdir(path)
files = sorted(files)

for f in files:
    if f != 'caso_clinico_radiologia1.txt':
        print(f)
    else:
        break

S0004-06142005000100009-1.txt
S0004-06142005000200004-1.txt
S0004-06142005000200009-1.txt
S0004-06142005000200009-2.txt
S0004-06142005000200009-3.txt
S0004-06142005000200009-4.txt
S0004-06142005000200009-5.txt
S0004-06142005000300011-1.txt
S0004-06142005000300012-1.txt
S0004-06142005000400011-1.txt
S0004-06142005000500011-1.txt
S0004-06142005000500014-1.txt
S0004-06142005000700014-1.txt
S0004-06142005000900012-1.txt
S0004-06142005000900013-1.txt
S0004-06142005000900014-1.txt
S0004-06142005000900015-1.txt
S0004-06142005000900016-1.txt
S0004-06142005000900017-1.txt
S0004-06142005001000010-1.txt
S0004-06142005001000011-1.txt
S0004-06142005001000011-2.txt
S0004-06142005001000011-3.txt
S0004-06142005001000014-1.txt
S0004-06142005001000015-1.txt
S0004-06142005001000016-1.txt
S0004-06142006000100010-1.txt
S0004-06142006000100011-1.txt
S0004-06142006000100012-1.txt
S0004-06142006000100013-1.txt
S0004-06142006000100014-1.txt
S0004-06142006000100015-1.txt
S0004-06142006000100015-2.txt
S0004-0614